In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
import numpy as np
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)

In [9]:
# Load the CSV into the DataFrame
streptococcaceae_without_pneumoniae_df = pd.read_csv(r"D:\Vivli 2024\Jupiter Notebooks\Vidata Analysis\Filtered Data for the Species and Families\streptococcaceae_without_pneumoniae_df.csv")

# Display the DataFrame (optional)
streptococcaceae_without_pneumoniae_df.head()


,Isolate Number,Data Source,Species,Family,Gender,Age Group,Country,Year,Source of Infection,Amoxicillin-clavulanate,Amoxicillin-clavulanate_I,Ampicillin,Ampicillin_I,Piperacillin tazobactam,Piperacillin tazobactam_I,Ceftriaxone,Ceftriaxone_I,Ceftazidime,Ceftazidime_I,Cefepime,Cefepime_I,Imipenem,Imipenem_I,Meropenem,Meropenem_I,Ciprofloxacin,Ciprofloxacin_I,Levofloxacin,Levofloxacin_I,Amikacin,Amikacin_I,Gentamicin,Gentamicin_I,Vancomycin,Vancomycin_I,Azithromycin,Azithromycin_I,Clarithromycin,Clarithromycin_I,Tigecycline,Tigecycline_I,Linezolid,Linezolid_I,Trimethoprim-sulfamethoxazole,Trimethoprim-sulfamethoxazole_I
0,1773292,ATLAS,Streptococcus agalactiae,Streptococcaceae,Female,19 to 64 Years,Spain,2018,Wound,NaN,NaN,NaN,NaN,<=0.25,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.03,NaN,1,NaN,NaN,NaN
1,1773293,ATLAS,Streptococcus pyogenes,Streptococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,NaN,NaN,NaN,<=0.25,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<=0.03,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.03,NaN,0.5,NaN,NaN,NaN
2,1773294,ATLAS,Streptococcus pyogenes,Streptococcaceae,Male,65 to 84 Years,Spain,2018,Wound,NaN,NaN,NaN,NaN,<=0.25,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<=0.03,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,0.03,NaN,1,NaN,NaN,NaN
3,1773295,ATLAS,Streptococcus pyogenes,Streptococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,NaN,NaN,NaN,<=0.25,NaN,<=0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<=0.03,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN,NaN,NaN,0.03,NaN,1,NaN,NaN,NaN
4,1773296,ATLAS,Streptococcus dysgalactiae,Streptococcaceae,Female,19 to 64 Years,Spain,2018,Wound,NaN,NaN,NaN,NaN,<=0.25,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<=0.03,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,0.03,NaN,1,NaN,NaN,NaN


In [10]:
# Function to extract numeric value from a string, handling cases like '<', '>', etc.
def extract_numeric_value(value):
    if isinstance(value, str):
        # Use regex to remove any non-numeric characters except for '.' and '-'
        cleaned_value = re.sub(r'[^\d.-]', '', value)
        try:
            return float(cleaned_value)
        except ValueError:
            return np.nan  # Return NaN if conversion fails
    return value  # Return the value as is if it's already numeric

In [11]:
# Function to interpret antibiotic values for Streptococcaceae (excluding Streptococcus pneumoniae)
def interpret_antibiotic_streptococcaceae_without_pneumoniae(row, antibiotic, year):
    value = extract_numeric_value(row[antibiotic])  # Clean the value

    # Handle missing or non-numeric values
    if pd.isna(value):
        return 'Use Not recommended'

    # Interpret for Penicillins (AMC, AMP, TZP)
    if antibiotic in ['Amoxicillin-clavulanate', 'Ampicillin', 'Piperacillin tazobactam']:
        if year in [2022, 2021, 2020, 2019, 2018]:
            return 'Susceptible' if value <= 0.25 else 'Resistant'

    # Interpret for Cephalosporins (AXO, PIP)
    elif antibiotic in ['Ceftriaxone', 'Cefepime']:
        if year in [2022, 2021, 2020, 2019, 2018]:
            return 'Susceptible' if value <= 0.25 else 'Resistant'

    # Interpret for Carbapenems (IPM, MEM)
    elif antibiotic in ['Imipenem', 'Meropenem']:
        if year in [2022, 2021, 2020, 2019, 2018]:
            return 'Susceptible' if value <= 0.25 else 'Resistant'

    # Interpret for Fluoroquinolones (LEV)
    elif antibiotic == 'Levofloxacin':
        if year in [2022, 2021, 2020]:
            if value <= 0.001:
                return 'Susceptible'
            elif 0.002 <= value <= 2:
                return 'Intermediate'
            else:
                return 'Resistant'
        elif year in [2018, 2019]:
            return 'Susceptible' if value <= 2 else 'Resistant'

    # Interpret for Glycopeptide (VAN)
    elif antibiotic == 'Vancomycin':
        if year in [2022, 2021, 2020, 2019, 2018]:
            return 'Susceptible' if value <= 2 else 'Resistant'

    # Interpret for Macrolides (AZM, CAM)
    elif antibiotic in ['Azithromycin', 'Clarithromycin']:
        if year in [2022, 2021, 2020, 2019, 2018]:
            if value <= 0.25:
                return 'Susceptible'
            elif 0.26 <= value <= 0.5:
                return 'Intermediate'
            else:
                return 'Resistant'

    # Interpret for Tetracyclines (TIG)
    elif antibiotic == 'Tigecycline':
        if year in [2022, 2021, 2020, 2019]:
            return 'Susceptible' if value <= 0.125 else 'Resistant'
        elif year == 2018:
            if value <= 0.25:
                return 'Susceptible'
            elif 0.26 <= value <= 0.5:
                return 'Intermediate'
            else:
                return 'Resistant'

    # Interpret for Oxazolidines (LZD)
    elif antibiotic == 'Linezolid':
        if year in [2022, 2021, 2020]:
            return 'Susceptible' if value <= 2 else 'Resistant'
        elif year in [2018, 2019]:
            if value <= 2:
                return 'Susceptible'
            elif 2.1 <= value <= 4:
                return 'Intermediate'
            else:
                return 'Resistant'

    # Interpret for Antifolate (SXT)
    elif antibiotic == 'Trimethoprim-sulfamethoxazole':
        if year in [2022, 2021, 2020, 2019, 2018]:
            if value <= 1:
                return 'Susceptible'
            elif 1.1 <= value <= 2:
                return 'Intermediate'
            else:
                return 'Resistant'

    return 'Use Not recommended'

# Function to apply MIC interpretation for Streptococcaceae (excluding Streptococcus pneumoniae)
def apply_interpretation_streptococcaceae_without_pneumoniae(row):
    antibiotics = [
        'Amoxicillin-clavulanate', 'Ampicillin', 'Piperacillin tazobactam', 'Ceftriaxone', 
        'Cefepime', 'Imipenem', 'Meropenem', 'Levofloxacin', 'Vancomycin', 
        'Azithromycin', 'Clarithromycin', 'Tigecycline', 'Linezolid', 'Trimethoprim-sulfamethoxazole'
    ]
    year = row['Year']
    
    for antibiotic in antibiotics:
        interpretation = interpret_antibiotic_streptococcaceae_without_pneumoniae(row, antibiotic, year)
        row[antibiotic + '_I'] = interpretation  # Fill the interpretation into the respective "_I" column
    return row

# Apply the interpretation across the filtered Streptococcaceae dataframe (excluding Streptococcus pneumoniae)
streptococcaceae_without_pneumoniae_df = streptococcaceae_without_pneumoniae_df.apply(apply_interpretation_streptococcaceae_without_pneumoniae, axis=1)

# Display the updated dataframe
streptococcaceae_without_pneumoniae_df.head()


,Isolate Number,Data Source,Species,Family,Gender,Age Group,Country,Year,Source of Infection,Amoxicillin-clavulanate,Amoxicillin-clavulanate_I,Ampicillin,Ampicillin_I,Piperacillin tazobactam,Piperacillin tazobactam_I,Ceftriaxone,Ceftriaxone_I,Ceftazidime,Ceftazidime_I,Cefepime,Cefepime_I,Imipenem,Imipenem_I,Meropenem,Meropenem_I,Ciprofloxacin,Ciprofloxacin_I,Levofloxacin,Levofloxacin_I,Amikacin,Amikacin_I,Gentamicin,Gentamicin_I,Vancomycin,Vancomycin_I,Azithromycin,Azithromycin_I,Clarithromycin,Clarithromycin_I,Tigecycline,Tigecycline_I,Linezolid,Linezolid_I,Trimethoprim-sulfamethoxazole,Trimethoprim-sulfamethoxazole_I
0,1773292,ATLAS,Streptococcus agalactiae,Streptococcaceae,Female,19 to 64 Years,Spain,2018,Wound,NaN,Use Not recommended,NaN,Use Not recommended,<=0.25,Susceptible,0.06,Susceptible,NaN,NaN,NaN,Use Not recommended,NaN,Use Not recommended,0.12,Susceptible,NaN,NaN,1,Susceptible,NaN,NaN,NaN,NaN,0.5,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.03,Susceptible,1,Susceptible,NaN,Use Not recommended
1,1773293,ATLAS,Streptococcus pyogenes,Streptococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,Use Not recommended,NaN,Use Not recommended,<=0.25,Susceptible,0.03,Susceptible,NaN,NaN,NaN,Use Not recommended,NaN,Use Not recommended,<=0.03,Susceptible,NaN,NaN,1,Susceptible,NaN,NaN,NaN,NaN,0.5,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.03,Susceptible,0.5,Susceptible,NaN,Use Not recommended
2,1773294,ATLAS,Streptococcus pyogenes,Streptococcaceae,Male,65 to 84 Years,Spain,2018,Wound,NaN,Use Not recommended,NaN,Use Not recommended,<=0.25,Susceptible,0.03,Susceptible,NaN,NaN,NaN,Use Not recommended,NaN,Use Not recommended,<=0.03,Susceptible,NaN,NaN,0.5,Susceptible,NaN,NaN,NaN,NaN,0.25,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.03,Susceptible,1,Susceptible,NaN,Use Not recommended
3,1773295,ATLAS,Streptococcus pyogenes,Streptococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,Use Not recommended,NaN,Use Not recommended,<=0.25,Susceptible,<=0.015,Susceptible,NaN,NaN,NaN,Use Not recommended,NaN,Use Not recommended,<=0.03,Susceptible,NaN,NaN,0.5,Susceptible,NaN,NaN,NaN,NaN,0.5,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.03,Susceptible,1,Susceptible,NaN,Use Not recommended
4,1773296,ATLAS,Streptococcus dysgalactiae,Streptococcaceae,Female,19 to 64 Years,Spain,2018,Wound,NaN,Use Not recommended,NaN,Use Not recommended,<=0.25,Susceptible,0.03,Susceptible,NaN,NaN,NaN,Use Not recommended,NaN,Use Not recommended,<=0.03,Susceptible,NaN,NaN,1,Susceptible,NaN,NaN,NaN,NaN,0.25,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.03,Susceptible,1,Susceptible,NaN,Use Not recommended


In [12]:
import pandas as pd

# List of species in Streptococcaceae family
streptococcaceae_species_list = [
    "Streptococcus pyogenes", 
    "Streptococcus agalactiae", 
    "Streptococcus dysgalactiae", 
    "Streptococcus canis"
]

# List of antibiotics of interest with interpretations (_I)
antibiotics_list = [
    'Amoxicillin-clavulanate_I', 'Ampicillin_I', 'Piperacillin tazobactam_I', 
    'Ceftriaxone_I', 'Ceftazidime_I', 'Cefepime_I', 'Imipenem_I', 'Meropenem_I', 
    'Ciprofloxacin_I', 'Levofloxacin_I', 'Amikacin_I', 'Gentamicin_I', 
    'Vancomycin_I', 'Azithromycin_I', 'Clarithromycin_I', 'Tigecycline_I', 
    'Linezolid_I', 'Trimethoprim-sulfamethoxazole_I'
]

# List of years of interest
years_list = [2018, 2019, 2020, 2021, 2022]

# Dictionary to store susceptibility results
streptococcaceae_results = {}

# Loop through each species of interest
for species in streptococcaceae_species_list:
    # Filter the data for the current species
    df_species = streptococcaceae_without_pneumoniae_df[streptococcaceae_without_pneumoniae_df['Species'] == species]

    # Dictionary to store antibiotic results
    antibiotic_results = {}

    # Loop through each antibiotic of interest
    for antibiotic in antibiotics_list:
        susceptibility_rates_list = []
        
        # Loop through each year
        for year in years_list:
            year_data = df_species[df_species['Year'] == year]
            total_count = len(year_data)
            
            if total_count > 0:
                # Calculate susceptibility counts
                susceptible_count = (year_data[antibiotic] == 'Susceptible').sum()
                intermediate_count = (year_data[antibiotic] == 'Intermediate').sum()
                resistant_count = (year_data[antibiotic] == 'Resistant').sum()

                # Calculate rates
                total_reported = susceptible_count + intermediate_count + resistant_count
                if total_reported > 0:
                    susceptible_rate = (susceptible_count / total_reported) * 100
                    intermediate_rate = (intermediate_count / total_reported) * 100
                    resistant_rate = (resistant_count / total_reported) * 100

                    # Round rates to 2 decimal places
                    susceptible_rate = round(susceptible_rate, 2)
                    intermediate_rate = round(intermediate_rate, 2)
                    resistant_rate = round(resistant_rate, 2)

                    # Append the data to the list
                    susceptibility_rates_list.append({
                        'Year': year,
                        'Susceptible': susceptible_rate,
                        'Intermediate': intermediate_rate,
                        'Resistant': resistant_rate
                    })
        
        # Store the results for this antibiotic
        if susceptibility_rates_list:
            antibiotic_results[antibiotic] = pd.DataFrame(susceptibility_rates_list)

    # Store the results for this species
    if antibiotic_results:
        streptococcaceae_results[species] = antibiotic_results

# Final results: Display susceptibility rates for Streptococcaceae grouped by years
for species, antibiotics_data in streptococcaceae_results.items():
    print(f"Results for Species: {species}")
    for antibiotic, df in antibiotics_data.items():
        print(f"\nSusceptibility rates for {antibiotic}:")
        print(df)
        print("\n" + "="*50 + "\n")


Results for Species: Streptococcus pyogenes

Susceptibility rates for Piperacillin tazobactam_I:
   Year  Susceptible  Intermediate  Resistant
0  2018        98.61           0.0       1.39
1  2019        98.84           0.0       1.16
2  2020        94.26           0.0       5.74
3  2021        98.35           0.0       1.65
4  2022        99.14           0.0       0.86



Susceptibility rates for Ceftriaxone_I:
   Year  Susceptible  Intermediate  Resistant
0  2018        99.87           0.0       0.13
1  2019       100.00           0.0       0.00
2  2020       100.00           0.0       0.00
3  2021        99.53           0.0       0.47
4  2022       100.00           0.0       0.00



Susceptibility rates for Meropenem_I:
   Year  Susceptible  Intermediate  Resistant
0  2018        99.87           0.0       0.13
1  2019       100.00           0.0       0.00
2  2020        99.84           0.0       0.16
3  2021        99.53           0.0       0.47
4  2022        99.57           0.0   

In [13]:
# Dictionary grouping antibiotics by their pharmacological classes
antibiotic_groups = {
    'Penicillins': ['Amoxicillin-clavulanate_I', 'Ampicillin_I', 'Piperacillin tazobactam_I'],
    'Cephalosporins': ['Ceftriaxone_I', 'Ceftazidime_I', 'Cefepime_I'],
    'Carbapenems': ['Imipenem_I', 'Meropenem_I'],
    'Fluoroquinolones': ['Ciprofloxacin_I', 'Levofloxacin_I'],
    'Aminoglycosides': ['Amikacin_I', 'Gentamicin_I'],
    'Glycopeptides': ['Vancomycin_I'],
    'Macrolides': ['Azithromycin_I', 'Clarithromycin_I'],
    'Tetracyclines': ['Tigecycline_I'],
    'Oxazolidinones': ['Linezolid_I'],
    'Antifolates': ['Trimethoprim-sulfamethoxazole_I']
}

# Loop through each species
for species, antibiotics_data in streptococcaceae_results.items():
    print(f"Visualizing results for Species: {species}")

    # Loop through each antibiotic group
    for group, antibiotics in antibiotic_groups.items():
        print(f"Visualizing results for Antibiotic Group: {group}")

        # Prepare a combined dataframe for the current group
        combined_df_list = []

        for antibiotic in antibiotics:
            if antibiotic in antibiotics_data:
                # Add the 'Antibiotic' column to identify each antibiotic in the group
                df = antibiotics_data[antibiotic].copy()
                df['Antibiotic'] = antibiotic.replace('_I', '')  # Removing the "_I" suffix for readability
                combined_df_list.append(df)

        # If there is data for this group, combine and plot
        if combined_df_list:
            combined_df = pd.concat(combined_df_list)

            # Plotting trends for each antibiotic in the group
            fig = px.line(
                combined_df,
                x='Year',
                y=['Susceptible', 'Intermediate', 'Resistant'],
                color='Antibiotic',  # Different colors for each antibiotic
                line_dash='variable',  # Different dash patterns for susceptible, intermediate, and resistant
                title=f"Susceptibility Trends for {species} ({group} Group)",
                labels={'value': 'Percentage (%)', 'variable': 'Resistance Category'},
                markers=True
            )

            # Customize the layout
            fig.update_layout(
                xaxis_title="Year",
                yaxis_title="Percentage (%)",
                legend_title="Antibiotic / Resistance Category",
                template="plotly",
                width=1000,  # Adjust the width
                height=600,  # Adjust the height
                xaxis=dict(tickmode='linear', tick0=2018, dtick=1)  # Ensure integer ticks for years
            )

            # Show the plot
            fig.show()


Visualizing results for Species: Streptococcus pyogenes
Visualizing results for Antibiotic Group: Penicillins


Visualizing results for Antibiotic Group: Cephalosporins


Visualizing results for Antibiotic Group: Carbapenems


Visualizing results for Antibiotic Group: Fluoroquinolones


Visualizing results for Antibiotic Group: Aminoglycosides
Visualizing results for Antibiotic Group: Glycopeptides


...

Visualizing results for Antibiotic Group: Fluoroquinolones


Visualizing results for Antibiotic Group: Aminoglycosides
Visualizing results for Antibiotic Group: Glycopeptides


Visualizing results for Antibiotic Group: Macrolides
Visualizing results for Antibiotic Group: Tetracyclines


Visualizing results for Antibiotic Group: Oxazolidinones


Visualizing results for Antibiotic Group: Antifolates


In [14]:
def check_increasing_trend(resistant_rates): 
    """
    This function checks if the resistance trend is increasing year over year.
    Returns True if resistance is increasing, False otherwise.
    """
    # Compare each element with the next one to ensure it's increasing
    for i in range(len(resistant_rates) - 1):
        if resistant_rates[i] > resistant_rates[i + 1]:
            return False  # If any rate decreases, return False
    return True  # Return True if all rates are increasing

def calculate_absolute_change(start_value, end_value):
    """
    This function calculates the absolute change between two values.
    """
    return end_value - start_value

# List to store results of increasing resistance trends
increasing_resistance_results = []

# Assuming streptococcaceae_results contains the data for Streptococcaceae
for species, antibiotics_data in streptococcaceae_results.items():
    # Loop through each antibiotic in the results
    for antibiotic, df in antibiotics_data.items():
        # Extract the resistant rates over the years
        resistant_rates = df['Resistant'].values
        years = df['Year'].values
        
        # Check if the resistance trend is increasing
        if check_increasing_trend(resistant_rates):
            # Calculate absolute changes between consecutive years
            absolute_changes = []
            for i in range(1, len(resistant_rates)):
                start_value = resistant_rates[i - 1]
                end_value = resistant_rates[i]
                absolute_change = calculate_absolute_change(start_value, end_value)
                absolute_changes.append(absolute_change)
            
            # Calculate the mean of the absolute changes
            if absolute_changes:
                mean_absolute_change = sum(absolute_changes) / len(absolute_changes)
            else:
                mean_absolute_change = 0.0
            
            # Store the information in the results
            increasing_resistance_results.append({
                'Species': species,
                'Antibiotic': antibiotic.replace('_I', ''),  # Remove "_I" for readability
                'Years': ', '.join(map(str, years)),  # Concatenate years for display
                'Resistant Rates': ', '.join(map(str, resistant_rates)),  # Concatenate rates for display
                'Overall Resistance Rate Increase': round(mean_absolute_change, 2)  # Round to 2 decimal places
            })

# Convert the results into a DataFrame for easy display
increasing_resistance_df = pd.DataFrame(increasing_resistance_results)

# Display the results
if not increasing_resistance_df.empty:
    print("Species and Antibiotics showing increasing resistance trends with Overall Resistance Rate Increase:")
    print(increasing_resistance_df.to_string(index=False))  # Display DataFrame without the index
else:
    print("No increasing resistance trends identified.")


Species and Antibiotics showing increasing resistance trends with Overall Resistance Rate Increase:
                   Species  Antibiotic                        Years         Resistant Rates  Overall Resistance Rate Increase
    Streptococcus pyogenes  Vancomycin 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
    Streptococcus pyogenes   Linezolid 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
  Streptococcus agalactiae  Vancomycin 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
  Streptococcus agalactiae   Linezolid 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
Streptococcus dysgalactiae Ceftriaxone 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
Streptococcus dysgalactiae   Meropenem 2018, 2019, 2020, 2021, 2022 0.0, 0.0, 0.0, 0.0, 0.0                               0.0
Streptococcus dysg